### 추천 시스템의 유형
1. **콘텐츠 기반 필터링** : 사용자가 특정한 아이템을 선호하는 경우 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식
2. **협업 필터링** : 사용자가 평가한 다른 아이템을 기반으로 사용자가 평가하지 않은 아이템의 예측 평가를 도출한다. 
    - **최근접 이웃 협업 필터링** : 사용자 행동 양식(평점정보, 상품구매이력)을 기반으로 추천하는 형태
        - '사용자-아이템의 평점행렬'에 기반하여 시작한다. 
        - 사용자 기반 : "당신과 비슷한 고객들도 다음 상품을 구매했습니다", 유사한 다른 사용자 Top-N을 선정해 추천하는 방식
        - 아이템 기반 : "이 상품을 선택한 다른 고객들도 다음 상품을 구매했습니다"
    - **잠재요인 협업 필터링** : 사용자-아이템 평점 매트릭스 속에 숨어있는 잠재 요인을 추출해 추천 예측을 할 수 있게 하는 기법
        - SVD와 같은 행렬분해를 기반으로 한다.
        - 다차원 희소행렬인 사용자-아이템 행렬 데이터를 저차원 밀집 행렬의 사용자-잠재요인 행렬과 아이템-잠재요인 전치행렬으로 분해
          할 수 있으며 이렇게 분해된 두 행렬의 내적을 통해 새로운 예측 사용자-아이템 평점 행렬 데이터를 만들어 예측이 가능하다.  
        - 확률적 경사하강법을 이용한 행렬분해 : P와 Q행렬로 계산된 예측 R행렬 값이 실제 행렬값과 최소의 오류를 가질 수 있도록 반복
          적인 비용함수의 최적화를 통해 P와 Q를 유추하는 것이다. 
    <br>
    
=> 넷플릭스 추천 시스템 경연 대회에서 행렬분해 기법을 이용한 잠재 요인 협업 필터링 방식이 우승하면서 온라인 스토어에서는 잠재요인 협업 필터링 기반의 추천 시스템이 작용한다. 요즘에는 개인화 특성을 강화한 하이브리드 형식으로 콘텐츠 기반화 협업 기반을 적절히 결합하여 사용하는 경우도 늘고 있다. 


### 9.5. 콘텐츠 기반의 필터링에 대한 실습 - TMDB 5000 영화 데이터 셋 활용
- 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성, 구성요소 등을 가진 다른 영화를 추천하는 방식이다.
- 영화를 추천하는데 가장 중요한 요소는 영화 장르 속성이므로 이를 기반으로 만든다. **장르 칼럼값의 유사도를 비교**한 후 그중 높은 평점을 가지는 영화를 추천하는 방식이다. 

In [1]:
#데이터 확인
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movies=pd.read_csv('tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [2]:
#콘텐츠 기반으로 분석할 때 필요한 열만 추출
movies_df=movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [6]:
#바꿔야 할 형태의 컬럼값 확인하기 
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...","[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp..."


In [4]:
#list[dict1, dict2]의 형태로 나올 수 있도록 변경하기 
from ast import literal_eval
movies_df['genres']=movies_df['genres'].apply(literal_eval)
movies_df['keywords']=movies_df['keywords'].apply(literal_eval)

In [5]:
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...","[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp..."


In [7]:
#genres내의 리스트에는 실제 장르만, keywords내의 리스트에는 실제 키워드만 들어가도록 작성하는 경우 
movies_df['genres']=movies_df['genres'].apply(lambda x: [y['name'] for y in x])
movies_df['keywords']=movies_df['keywords'].apply(lambda x:[y['name'] for y in x])
movies_df[['genres', 'keywords']][:1] #확인

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [10]:
#장르 콘텐츠 유사도 측정
# 문자열로 이루어진 장르를 counts기반의 피처벡터화 이후에 코사인 유사도를 측정하여 출력한다. 
# 장르 유사도가 높은 영화 중에서 평점이 높은 순으로 영화를 추천한다. 
from sklearn.feature_extraction.text import CountVectorizer

#Countvectorizer를 적용하기 위해 공백문자로 word단위가 구분되는 문자열로 변환한다. 
movies_df['genres_literal']=movies_df['genres'].apply(lambda x:(' ').join(x))
count_vect=CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat=count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape) #4803개의 레코드와 276개의 개별 단어로 이루어진 벡터 행렬이 만들어 졌다.

(4803, 276)


In [12]:
#만들어진 벡터행렬에 대해 코사인 유사도를 계산한다
from sklearn.metrics.pairwise import cosine_similarity

genre_sim=cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [13]:
#콘텐츠 기반의 필터링을 수행하려면 movie_df의 개별 레코드에 대해 가장 장르 유사도가 높은 순으로 다른 레코드를 추출해야 한다, 
genre_sim_sorted_ind=genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


In [16]:
#이제 genre_sim_sorted_ind으로 특정 값에 대해 유사도가 높은 인덱스를 뽑을 수 있게 되었다. 
#이제 장르 콘텐츠 필터링을 통한 영화를 추천한다. 
#고객이 선정한 추천 기준이 되는 영화제목, 추천할 영화 건수를 입력하면 추천 영화 정보를 가지는 df를 반환한다. 
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    #인자로 입력된 movie_df에서 'title'컬럼이 입력된 값에 대한 df를 출력한다
    title_movie=df[df['title']==title_name]
    
    #title_named를 가진 dataframe의 index객체를 ndarray로 반환하고 
    #sorted_ind인자로 입력된 genre_sim_sorted_int객체에서 유사도 순으로 top_n개의 index를 추출
    title_index=title_movie.index.values
    similar_indexes=sorted_ind[title_index, :(top_n)]
    
    #추출된 top_n index를 추출, top_n index는 2차원 데이터임
    #df에서 index로 사용하기 위해 1차원 array로 변경
    print(similar_indexes)
    similar_indexes=similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [17]:
#영화 '대부'와 장르별로 유사한 영화 10개를 한번 추천해보자 
similar_movies=find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [18]:
#평점이 0인 영화도 있으므로 다시 평점 순서대로 필터링해서 최종 결정해본다. (top10추출)
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [19]:
#평점에 대해 vote_count를 고려해본다. 1명이 평가했으면 신뢰성이 떨어진다. 
#평가 횟수를 반영할 수 있는 새로운 평가 방식이 필요하다
#가중평점이라는 새로운 변수를 생성한다. 582p의 식참고 
C=movies_df['vote_average'].mean() #전체 영화 평점
m=movies_df['vote_count'].quantile(0.6) #전체 투표횟수에서 60%에 해당하는 값
print('C:', round(C,3), 'm:', round(m,3))

C: 6.092 m: 370.2


In [32]:
#기존평점을 새로운 가중평점으로 재정의하여 대입하고 새로운 컬럼을 생성하여 대입한다. 
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()
#새로운 가중치 값
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C ) 

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) #새롭게 재정의된 평점평균값 

In [33]:
#새롭게 부여된 weighted_vote의 평점이 높은 순으로 10개를 추출
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [34]:
#적당히 원하는 결과 도출됨
# 다시 '대부'영화에 대해 유사한 영화 콘텐츠기반 필터링 방식을 사용한다. 
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
# 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


하지만 장르 만으로 영화가 전달하는 많은 분위기와 요소, 개인이 좋아하는 성향을 모두 반영했다고 하기는 어렵다. 다른 콘텐츠를 기반으로 확장해 추천시스템을 고도화는 부분을 생각할 필요가 있다. 


### 9.6. 아이템 기반 최근접 이웃 협업 필터링 실습
- 최근접 이웃 현업 필터링은 사용자 기반과 아이템 기반으로 분류된다. 이 중에서 일반적으로 추천 정확도가 뛰어난 아이템 기반의 협업 필터링을 구현한다 
- 단순히 영화간의 유사도를 구분하여 영화를 추천하는 방법도 가능하나, '개인화'된 방법은 아니다
- 이에 따라 아이템 기반 최근접 이웃 협업 필터링을 사용하여 개인화된 평점을 매겨 이를 기준으로 분류하도록 한다. 
- 협업 필터링은 rating.csv와 같이 사용자와 아이템 간의 평점에 기반해 영화를 추천하는 시스템이다 

In [35]:
import pandas as pd
import numpy as np
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')
print(movies.shape)
print(ratings.shape)

(58098, 3)
(27753444, 4)


In [41]:
#사용자 로우, 영화 컬럼으로 구성된 사용자-영화 평점 데이터를 생성한다. 
ratings=ratings[['userId', 'movieId', 'rating']][:10000] #데이터크기 조정(너무 커서 안돌아가므로 )
ratings_matrix=ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix.head(3)

movieId,1,2,5,6,10,11,16,19,20,23,...,2840,2915,2986,3020,3171,3363,3424,3698,3826,3893
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,2.5,4.0,NaN,NaN,4.5,3.5,2.0,3.5
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN


In [43]:
#movies df와 조인하여 movie id를 movie title로 매핑하여 작성한다. 
ratings_movies=pd.merge(ratings, movies, on='movieId')

#columns='title'로 title컬럼으로 피벗 수행
ratings_matrix=ratings_movies.pivot_table('rating', index='userId', columns='title')

In [44]:
#NaN값은 모두 0으로 변환한다. 
ratings_matrix=ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,Ace Ventura: When Nature Calls (1995),"Adventures of Pinocchio, The (1996)",American Graffiti (1973),"American President, The (1995)","American Werewolf in London, An (1981)",Angel on My Shoulder (1946),Angels and Insects (1995),Apollo 13 (1995),Arlington Road (1999),Assassins (1995),...,Three Colors: Blue (Trois couleurs: Bleu) (1993),To Die For (1995),Toy Story (1995),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Under Siege 2: Dark Territory (1995),"Usual Suspects, The (1995)",Waiting for Guffman (1996),Weekend at Bernie's (1989),Weird Science (1985),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,0.0,0.0,0.0,0.0,0.0,4.5,1.5,4.5,4.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,4.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
#영화간 유사도 산출
# 사용자-영화 평점 행렬 데이터 셋을 이용하여 영화간의 유사도를 측정한다. (평점을 기준으로 영화 유사도를 평가한다, )
rating_matrix_T=ratings_matrix.transpose()
rating_matrix_T.head(3)

userId,1,2,3,4
title,,,,
Ace Ventura: When Nature Calls (1995),0.0,0.0,0.0,2.0
"Adventures of Pinocchio, The (1996)",0.0,0.0,4.0,0.0
American Graffiti (1973),0.0,4.0,0.0,0.0


In [50]:
#영화간의 코사인 유사도를 구하고 영화 제목과 매핑한다. 
from sklearn.metrics.pairwise import cosine_similarity

item_sim=cosine_similarity(rating_matrix_T, rating_matrix_T)

#cosine_similarity()로 반환된 넘파이 행렬을 영화명으로 매핑하여 df로 변환
item_sim_df=pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(99, 99)


title,Ace Ventura: When Nature Calls (1995),"Adventures of Pinocchio, The (1996)",American Graffiti (1973),"American President, The (1995)","American Werewolf in London, An (1981)",Angel on My Shoulder (1946),Angels and Insects (1995),Apollo 13 (1995),Arlington Road (1999),Assassins (1995),...,Three Colors: Blue (Trois couleurs: Bleu) (1993),To Die For (1995),Toy Story (1995),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Under Siege 2: Dark Territory (1995),"Usual Suspects, The (1995)",Waiting for Guffman (1996),Weekend at Bernie's (1989),Weird Science (1985),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
Ace Ventura: When Nature Calls (1995),1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
"Adventures of Pinocchio, The (1996)",0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
American Graffiti (1973),0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
#아이템기반 최근접 이웃 협업 필터링으로 개인화된 영화를 추천하는 경우 
#위의 영화간 유사도만 가지고 영화를 추천하면 개인적인 성향은 반영되지 않음
#개인화된 예측 평점식 (591p를 참고)를 유사도 벡터값인 높은 영화, 실제 평점 벡터를 가지로 재정의한 값 

#사용자별 예측 평점 
def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [58]:
#위의 predict_rating함수를 이용하여 실제 개인화된 평점을 구해본다.
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,Ace Ventura: When Nature Calls (1995),"Adventures of Pinocchio, The (1996)",American Graffiti (1973),"American President, The (1995)","American Werewolf in London, An (1981)",Angel on My Shoulder (1946),Angels and Insects (1995),Apollo 13 (1995),Arlington Road (1999),Assassins (1995),...,Three Colors: Blue (Trois couleurs: Bleu) (1993),To Die For (1995),Toy Story (1995),Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Under Siege 2: Dark Territory (1995),"Usual Suspects, The (1995)",Waiting for Guffman (1996),Weekend at Bernie's (1989),Weird Science (1985),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000,...,3.3125,0.00000,0.00000,0.00000,0.00000,0.00000,3.3125,3.3125,3.3125,3.3125
2,0.03951,0.000000,3.669385,0.03951,0.000000,0.000000,0.03951,0.03951,3.669385,0.03951,...,0.0000,0.03951,0.03951,0.03951,0.03951,0.03951,0.0000,0.0000,0.0000,0.0000
3,0.00000,3.545455,0.000000,0.00000,3.545455,3.545455,0.00000,0.00000,0.000000,0.00000,...,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.0000,0.0000


In [59]:
#유사도 내적을 거치기 떄문에 실제 값과의 차이는 있을 수 밖에 없다
#우리의 목적은 그 오차를 '줄이는 것'이다. 
#예측을 평가하는 mse값을 만들어 줄이도록 해본다.
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

아이템 기반 모든 인접 이웃 MSE:  1.3493416807702445


In [60]:
#많은 영화의 유사도 벡터를 계산하니 평점 예측이 떨어진다
#특정 영화와 비슷한 유사도를 가지는 영화에 대해서만 유사도 벡터를 적용하도록 한다. 
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred

In [61]:
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))


# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

아이템 기반 인접 TOP-20 이웃 MSE:  1.3347704088544907


In [64]:
#안본 영화에 대한 평점을 예측하여 추천하도록 하는 함수 작성 
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

```python
#추천하는 함수 작성
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies
    
# 사용자가 관람하지 않는 영화명 추출   
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies
```

### 9.7. 행렬 분해를 이용한 잠재 요인 협업 필터링 실습
- 행렬분해를 이용한 잠재요인 필터링을 구현한다. 일반적인 행렬분해에는 SVD가 자주 사용되지만 희소행렬이 대부분인 경우가 많기 때문에 SGD기반의 행렬 분해를 구현하고 이를 기반으로 사용자에게 영화를 추천한다. 

In [72]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [66]:
#SGD기반의 행렬 분해를 구현하기 위한 matrix_factorization함수를 구한다. 
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트. 
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

In [70]:
#새롭게 데이터를 불러오고 사용자-아이템 평점 행렬화 한다. 
import pandas as pd
import numpy as np

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')[:10000]
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

# title 컬럼을 얻기 이해 movies 와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

In [73]:
#만들어진 행렬을 기반으로 행렬 분해를 수행한다
#step=200, 잠재요인 차원은 50, 학습률과 L2Regularization 계수는 0.01로 둔다. 
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse :  3.719347406819938
### iteration step :  10  rmse :  0.8397284016272849
### iteration step :  20  rmse :  0.4757959062425241
### iteration step :  30  rmse :  0.30346287167599195
### iteration step :  40  rmse :  0.20892177963002978
### iteration step :  50  rmse :  0.15359132930936506
### iteration step :  60  rmse :  0.11882479329494486
### iteration step :  70  rmse :  0.09683668924237193
### iteration step :  80  rmse :  0.08256204101962378
### iteration step :  90  rmse :  0.07289108007468449
### iteration step :  100  rmse :  0.06604236303582803
### iteration step :  110  rmse :  0.06100095976437129
### iteration step :  120  rmse :  0.05716817158949802
### iteration step :  130  rmse :  0.05417364214727288
### iteration step :  140  rmse :  0.051777836346029144
### iteration step :  150  rmse :  0.049820264978628334
### iteration step :  160  rmse :  0.048190547640328776
### iteration step :  170  rmse :  0.04681121473352206
### iteration step :  

In [74]:
#더 나은 시각화를 위해 영화 제목과 mapping한다. 
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

ratings_pred_matrix.head(3)

title,(500) Days of Summer (2009),*batteries not included (1987),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),"11th Hour, The (2007)",12 Angry Men (1957),...,Zeus and Roxanne (1997),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),"Zorro, the Gay Blade (1981)",eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.392798,3.775178,2.165679,2.849254,1.842920,3.467683,2.808213,2.665300,2.334163,4.279206,...,4.396554,4.316504,3.854754,2.138178,4.604164,0.820860,2.693138,1.530667,0.349258,3.953208
2,3.053467,3.878127,2.176524,2.833102,2.020025,3.103925,2.505167,2.766184,2.381297,4.108037,...,4.371414,4.076968,3.697622,2.207712,4.650280,0.861523,2.822128,1.736894,0.331655,3.291903
3,3.783394,3.856312,2.128497,2.789990,2.286300,3.017945,2.406882,2.678489,2.379869,3.889922,...,4.466368,4.092889,3.701543,2.403998,4.694544,0.864986,2.816587,1.791948,0.330083,3.626151


In [75]:
#사용자가 보지 않은 영화에 대한 예측하는 함수의 작성 
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

In [76]:
#추천 시스템 
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [77]:

# 사용자가 관람하지 않는 영화명 추출   
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
"English Patient, The (1996)",3.503416
"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",3.483805
Life Is Beautiful (La Vita è bella) (1997),3.426615
Taxi Driver (1976),3.408768
One Flew Over the Cuckoo's Nest (1975),3.365581
Fight Club (1999),3.355224
"Matrix, The (1999)",3.316472
Star Wars: Episode V - The Empire Strikes Back (1980),3.271957
"Shawshank Redemption, The (1994)",3.255316


### 9.8.python의 추천 시스템 패키지의 Surprise를 사용해 볼 수도 있다 